<a href="https://colab.research.google.com/github/amina04/CNN-lstm/blob/main/CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [401]:
from google.colab import drive
from sklearn.preprocessing import MinMaxScaler

In [402]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


la bibliothéque pandas nous donne le droit de travailler avec la format csv

In [403]:
import pandas as pd  

header sert a ajouter les indices ou data pour ne pas faire le 1 er row comme indice

In [404]:

featureV=["duration","protocol_type","service","flag","src_bytes","dst_bytes","land","wrong_fragment","urgent","hot",
          "num_failed_logins","logged_in","num_compromised","root_shell","su_attempted","num_root","num_file_creations","num_shells",
          "num_access_files","num_outbound_cmds","is_host_login","is_guest_login","count","srv_count","serror_rate","srv_serror_rate",
          "rerror_rate","srv_rerror_rate","same_srv_rate","diff_srv_rate","srv_diff_host_rate","dst_host_count","dst_host_srv_count", 
          "dst_host_same_srv_rate","dst_host_diff_srv_rate","dst_host_same_src_port_rate","dst_host_srv_diff_host_rate","dst_host_serror_rate",
          "dst_host_srv_serror_rate","dst_host_rerror_rate","dst_host_srv_rerror_rate","label","difficulty"]

flagV=['OTH','RSTOS0','SF','SH','RSTO','S2','S1','REJ','S3','RSTR','S0']

protocol_typeV=['tcp','udp','icmp']

serviceV=['http','smtp','finger','domain_u','auth','telnet','ftp','eco_i','ntp_u','ecr_i','other','private','pop_3','ftp_data',
                  'rje','time','mtp','link','remote_job','gopher','ssh','name','whois','domain','login','imap4','daytime','ctf','nntp',
                  'shell','IRC','nnsp','http_443','exec','printer','efs','courier','uucp','klogin','kshell','echo','discard','systat',
                  'supdup','iso_tsap','hostnames','csnet_ns','pop_2','sunrpc','uucp_path','netbios_ns','netbios_ssn','netbios_dgm',
                  'sql_net','vmnet','bgp','Z39_50','ldap','netstat','urh_i','X11','urp_i','pm_dump','tftp_u','tim_i','red_i','icmp',
                  'http_2784','harvest','aol','http_8001']

binary_attack=['normal','ipsweep', 'nmap', 'portsweep','satan', 'saint', 'mscan','back', 'land', 'neptune', 'pod', 'smurf',
               'teardrop', 'apache2', 'udpstorm', 'processtable','mailbomb','buffer_overflow', 'loadmodule', 'perl', 'rootkit',
               'xterm', 'ps', 'sqlattack','ftp_write', 'guess_passwd', 'imap', 'multihop','phf', 'spy', 'warezclient',
               'warezmaster','snmpgetattack','named', 'xlock', 'xsnoop','sendmail', 'httptunnel', 'worm', 'snmpguess']

multiclass_attack={ 'normal': 'normal',
        'probe': ['ipsweep.', 'nmap.', 'portsweep.','satan.', 'saint.', 'mscan.'],
        'dos': ['back.', 'land.', 'neptune.', 'pod.', 'smurf.','teardrop.', 'apache2.', 'udpstorm.', 'processtable.','mailbomb.'],
        'u2r': ['buffer_overflow.', 'loadmodule.', 'perl.', 'rootkit.','xterm.', 'ps.', 'sqlattack.'],
        'r2l': ['ftp_write.', 'guess_passwd.', 'imap.', 'multihop.','phf.', 'spy.', 'warezclient.', 'warezmaster.','snmpgetattack.',
                   'named.', 'xlock.', 'xsnoop.','sendmail.', 'httptunnel.', 'worm.', 'snmpguess.']}    

In [405]:
data_train=pd.read_csv('/content/drive/MyDrive/data/nsl-kdd/KDDTrain+.txt',names=featureV)
data_valid=pd.read_csv('/content/drive/MyDrive/data/nsl-kdd/KDDTest+.txt',names=featureV)


In [406]:
data_train.shape

(125973, 43)

In [407]:
data_valid.shape


(22544, 43)

In [408]:
data_train = data_train.query("service != 'aol'")
data_train = data_train.query("service != 'harvest'")
data_train = data_train.query("service != 'http_2784'")
data_train = data_train.query("service != 'http_8001'")
data_train = data_train.query("service != 'red_i'")
data_train = data_train.query("service != 'urh_i'")
data_train = data_train.query("service != 'printer'")
data_train = data_train.query("service != 'rje'")
#-------------------------------------------------------------------------------->>>>
data_valid = data_valid.query("service != 'printer'")
data_valid = data_valid.query("service != 'rje'")



## **Working with Validation data -Numericalization-**


In [409]:
#data=données cls=classe binary or multiclass   df=train or test string
def preprocessing(data,cls,df): 
  #----------attack categorization----------------->
  data['label']=data['label'].replace(['normal.','normal'],0)
  #----------------------binary classification--------------------->
  if cls=='binary':
    for i in range(len(binary_attack)):
      data['label'] = data['label'].replace(binary_attack[i], 1)
  #-------------------------splitting features and labels---------------->
  y=data['label']
  #partier de dataset qui contien les strings
  x=data.loc[:,'duration':'hot']
  #-----------------converting to binary feature vectors-------------------------------------->  
  #on va copier column dans un variable 
  
  t=x.protocol_type.copy()
  #puis utiliser la fonction dumies qui transfer en binaire
  t=pd.get_dummies(t)
  #on va supprimer column l original qui contient string 1 ie tout rows de protocole type 
  x=x.drop(columns='protocol_type',axis=1)
  #ajouter nouveau column qui contient binaire
  x=x.join(t)

  t1=x.service.copy()
  t1=pd.get_dummies(t1)
  x=x.drop(columns='service',axis=1)
  x=x.join(t1)

  t2=x.flag.copy()
  t2=pd.get_dummies(t2)
  x=x.drop(columns='flag',axis=1)
  x=x.join(t2)
#mettre tout les valeurs entre 0 et 1  normalization
  x = MinMaxScaler(feature_range=(0, 1)).fit_transform(x)
  #------------------------------------------------------------------------------>
  #-----------------converting to binary label vectors  train--------------------------------------> 
  yt=y.copy()
  yt=pd.get_dummies(yt)
#------------------------------------------------------------------------------>
  return x,yt
  #if df=='train':
    #retourner les val et label
   # return x,yt
 # else:
  #  return x,yt

In [410]:
#le 1 er var sera pour features et 2em pour label
x_valid,y_valid=preprocessing(data_valid,cls='binary',df='valid')
x_train,y_train=preprocessing(data_train,cls='binary',df='train')
y_train.shape


(125793, 2)

transformer de 1D-->2D car cnn travaille avec 2D 
train (83,1)

In [411]:
import numpy as np
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
x_train.shape
#x_train=x_train.reshape(-1,13,7,1)
#x_train.shape




(125793, 83, 1)

In [412]:
x_valid = np.reshape(x_valid, (x_valid.shape[0], x_valid.shape[1], 1))
x_valid.shape



(22525, 83, 1)

# **Création de model**



In [413]:
#instancier le modéle
from tensorflow.keras.models import Sequential
model=Sequential()
#Créer la couche entrée totalement connecté avec la couche dense
from tensorflow.keras.layers import Conv1D, Dense, Dropout, Flatten, MaxPooling1D , AveragePooling1D,BatchNormalization
from tensorflow.keras.layers import LeakyReLU



In [414]:
#model.add(Conv1D(32,3, padding="same",activation="relu",input_shape = (x_train.shape[1], 1)))
model.add(Conv1D(32,kernel_size =5,padding="same",input_shape = (x_train.shape[1], 1) ))
model.add(LeakyReLU(alpha=0.1))
model.add(BatchNormalization())
model.add(AveragePooling1D(pool_size=2))
model.add(Dropout(0.05))
#model.add(MaxPooling1D(pool_size=(4)))
model.add(Flatten())
#model.add(Dropout(0.5))
#(83*32)/4=664
model.add(Dense(units=664))
model.add(LeakyReLU(alpha=0.1))
model.add(Dense(2, activation="softmax"))

In [415]:
model.summary()

Model: "sequential_25"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_25 (Conv1D)          (None, 83, 32)            192       
                                                                 
 leaky_re_lu_49 (LeakyReLU)  (None, 83, 32)            0         
                                                                 
 batch_normalization_25 (Bat  (None, 83, 32)           128       
 chNormalization)                                                
                                                                 
 average_pooling1d_25 (Avera  (None, 41, 32)           0         
 gePooling1D)                                                    
                                                                 
 dropout_25 (Dropout)        (None, 41, 32)            0         
                                                                 
 flatten_25 (Flatten)        (None, 1312)            

**The learning rate** is a hyperparameter that controls how much to change the model in response to the estimated error each time the model weights are updated. Choosing the learning rate is challenging as a value too small may result in a long training process that could get stuck, whereas a value too large may result in learning a sub-optimal set of weights too fast or an unstable training process.

The learning rate may be the most important hyperparameter when configuring your neural network. Therefore it is vital to know how to investigate the effects of the learning rate on model performance and to build an intuition about the dynamics of the learning rate on model behavior.

In [ ]:
from tensorflow import keras
#opt = keras.optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer ='adam',loss = 'categorical_crossentropy', metrics = ['accuracy',keras.metrics.Precision()])
#model.fit(x_train, y_train, epochs = 100, batch_size =1000)
model.fit(x_train, y_train, epochs =85,batch_size =128,validation_data=(x_valid,y_valid))

Epoch 1/85
973/983 [============================>.] - ETA: 0s - loss: 0.1236 - accuracy: 0.9599 - precision_19: 0.9599